# [Autogen 0.5.5 (stable) setup](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/installation.html)
## Install OpenAI for Model Client
- To use the OpenAI and Azure OpenAI models, you need to install the following extensions: `pip install "autogen-ext[openai]"`
- If you are using Azure OpenAI with AAD authentication, you need to install the following: `pip install "autogen-ext[azure]"`
- If you want to install both libraries, you maysimply install the following: `pip install "autogen-ext[openai,azure]"`


## Full setup
```
conda env remove -n autogen -y
conda create -n autogen python=3.13 -y
conda activate autogen
pip install -U "autogen-agentchat" "autogen-ext[azure]" "autogen-ext[openai]" python-dotenv jupyter
jupyter kernelspec list
jupyter kernelspec uninstall autogen -y
jupyter kernelspec list
python -m ipykernel install --name autogen --user
```
Other possible libraries: *pyautogen markdownify openai chroma apify-client duckduckgo-search langchain langchain-community*

# Libaries and Constants

In [1]:
# !az login

In [2]:
import os, sys
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:    
    print("Environment variables have been loaded ;-)")

Environment variables have been loaded ;-)


# Define a model client

In [3]:
def create_model_client(aad:bool=True):
    if aad:
        from autogen_ext.auth.azure import AzureTokenProvider
        from azure.identity import DefaultAzureCredential
        
        token_provider = AzureTokenProvider(
            DefaultAzureCredential(), 
            "https://cognitiveservices.azure.com/.default")
        
        model_client = AzureOpenAIChatCompletionClient(
            azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
            azure_deployment = os.environ["MODEL_DEPLOYMENT_NAME"],
            model = os.environ["MODEL_DEPLOYMENT_NAME"], # this is the model of the deployment
            api_version = os.environ["OPENAI_API_VERSION"],            
            azure_ad_token_provider=token_provider,
        )
    else:
        model_client = AzureOpenAIChatCompletionClient(
            azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
            model = os.environ["MODEL_DEPLOYMENT_NAME"], # this is the model of the deployment
            api_version = os.environ["OPENAI_API_VERSION"],
            api_key=os.environ["AZURE_OPENAI_API_KEY"],
        )

    return model_client
    
model_client = create_model_client(aad=True)
model_client.dump_component()

ComponentModel(provider='autogen_ext.models.openai.AzureOpenAIChatCompletionClient', component_type='model', version=1, component_version=1, description='Chat completion client for Azure OpenAI hosted models.', label='AzureOpenAIChatCompletionClient', config={'model': 'gpt-4o', 'azure_endpoint': 'https://mmoaiswc-01.openai.azure.com/', 'azure_deployment': 'gpt-4o', 'api_version': '2025-03-01-preview', 'azure_ad_token_provider': {'provider': 'autogen_ext.auth.azure.AzureTokenProvider', 'component_type': 'token_provider', 'version': 1, 'component_version': 1, 'label': 'AzureTokenProvider', 'config': {'provider_kind': 'DefaultAzureCredential', 'scopes': ['https://cognitiveservices.azure.com/.default']}}})

# Define a simple function tool that the agent can use

In [4]:
# For this example, we use a fake weather tool for demonstration purposes

async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 23 Celsius degrees and Sunny"


await get_weather("non-existing city")

'The weather in non-existing city is 23 Celsius degrees and Sunny'

# Define an AssistantAgent with:
- model
- tool
- system message
- reflection enabled

In [5]:
# The system message instructs the agent via natural language
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

# Run the agent and stream the messages to the console

In [6]:
async def main(question:str, terminate:bool=False) -> None:
    await Console(agent.run_stream(task=question))
    if terminate: # Close the connection to the model client.
        await model_client.close()    

In [9]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main(question = "Che tempo fa nel più grande comune di Milano, esclusa Milano?")

---------- TextMessage (user) ----------
Che tempo fa nel più grande comune di Milano, esclusa Milano?
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
Il più grande comune della provincia di Milano, escluso Milano, è Sesto San Giovanni. Procedo a controllare il meteo.
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_DODb4FniuGp3Kddm6SsmUSNO', arguments='{"city":"Sesto San Giovanni"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in Sesto San Giovanni is 23 Celsius degrees and Sunny', name='get_weather', call_id='call_DODb4FniuGp3Kddm6SsmUSNO', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
A Sesto San Giovanni ci sono 23 gradi Celsius ed è soleggiato.
